In [37]:
import pandas as pd
from datetime import datetime, timedelta

caminho = "/content/[PS][Desafio] Excel - Desafio Excel 1.csv"

df = pd.read_csv(caminho, sep=',')

print(df.head())


           Nome Completo             Telefone              Mensagem Recebida  \
0           Alexia Viana     +55 31 5869-6342                            NaN   
1  João Gabriel da Rocha     +55 51 6760 9107                            NaN   
2     Ana Julia Oliveira        0300-025-2534      Não me respondeu ainda...   
3     Dr. Otávio da Mata  +55 (081) 8357 7893  Olá, qual o valor do serviço?   
4            Joana Viana         84 5570 8039               Tem vaga amanhã?   

  Data do Último Contato  Status  
0             02/05/2025     NaN  
1             25/04/2025     NaN  
2             24/04/2025     NaN  
3             07/05/2025     NaN  
4             29/04/2025     NaN  


In [39]:
#removendo o que nao é numero, 55 no início e telemarketing

from os import remove
df['Telefone'] = df['Telefone'].str.replace(r'\D+', '', regex=True)
df['Telefone'] = df['Telefone'].str.replace(r'^55', '', regex=True)
df.loc[df['Telefone'].str.startswith(('0300', '0800', '0900', '0500')), 'Telefone'] = None
print(df.head(10))





           Nome Completo     Telefone              Mensagem Recebida  \
0           Alexia Viana   3158696342                            NaN   
1  João Gabriel da Rocha   5167609107                            NaN   
2     Ana Julia Oliveira         None      Não me respondeu ainda...   
3     Dr. Otávio da Mata  08183577893  Olá, qual o valor do serviço?   
4            Joana Viana   8455708039               Tem vaga amanhã?   
5   Maria Vitória Fogaça  03148516533                            NaN   
6       Davi Lucas Souza         None  Olá, qual o valor do serviço?   
7   Ana Carolina da Mata  06104341898      Não me respondeu ainda...   
8          Marcela Pinto  08495812347      Não me respondeu ainda...   
9        Bárbara da Rosa  04120121243      Não me respondeu ainda...   

  Data do Último Contato  Status  
0             02/05/2025     NaN  
1             25/04/2025     NaN  
2             24/04/2025     NaN  
3             07/05/2025     NaN  
4             29/04/2025     NaN

In [40]:
#padroniza os DDDs

df['Telefone'] = df['Telefone'].str.replace(r'^0', '', regex=True)


In [41]:
#Aplicando o 9 na frente dos numeros que tem 10 digits (DDD + 8)

df['Telefone'] = df["Telefone"].apply(
    lambda x: x[:2] + '9' + x[2:] if isinstance(x, str) and len(x) == 10 else x
)

In [42]:
#Acrescentando o +55 na frente dos números que tem 11 digits(DDD + 9)

df['Telefone'] =df['Telefone'].apply(
    lambda x: '+55' + x if isinstance(x, str) and len(x) == 11 else x
)

In [43]:
#Conferindo se só existem valores formatados ou nulos

print(df.loc[df['Telefone'].notna() & (df['Telefone'].str.len() != 14)])

Empty DataFrame
Columns: [Nome Completo, Telefone, Mensagem Recebida, Data do Último Contato, Status]
Index: []


In [44]:
#Telefone padronizado

print(df.head(5))

           Nome Completo        Telefone              Mensagem Recebida  \
0           Alexia Viana  +5531958696342                            NaN   
1  João Gabriel da Rocha  +5551967609107                            NaN   
2     Ana Julia Oliveira            None      Não me respondeu ainda...   
3     Dr. Otávio da Mata  +5581983577893  Olá, qual o valor do serviço?   
4            Joana Viana  +5584955708039               Tem vaga amanhã?   

  Data do Último Contato  Status  
0             02/05/2025     NaN  
1             25/04/2025     NaN  
2             24/04/2025     NaN  
3             07/05/2025     NaN  
4             29/04/2025     NaN  


In [45]:
df.rename(columns={'Contato Padronizado' : 'Contato Ativo?'}, inplace =True)

In [49]:
df['Data do Último Contato'] = pd.to_datetime(df['Data do Último Contato'], format='%d/%m/%Y')
limite = pd.Timestamp.today() - pd.Timedelta(days=5)


In [52]:
#Filtro para contato ativo se nao for nulo e tiver menos de 5 dias

df['Contato Ativo?'] = df.apply(
    lambda row: 'Sim' if pd.notna(row['Mensagem Recebida']) and row['Data do Último Contato'] >= limite else 'Não',
    axis = 1
)

In [53]:
print(df.head())

           Nome Completo        Telefone              Mensagem Recebida  \
0           Alexia Viana  +5531958696342                            NaN   
1  João Gabriel da Rocha  +5551967609107                            NaN   
2     Ana Julia Oliveira            None      Não me respondeu ainda...   
3     Dr. Otávio da Mata  +5581983577893  Olá, qual o valor do serviço?   
4            Joana Viana  +5584955708039               Tem vaga amanhã?   

  Data do Último Contato  Status Contato Ativo?  
0             2025-05-02     NaN            Não  
1             2025-04-25     NaN            Não  
2             2025-04-24     NaN            Não  
3             2025-05-07     NaN            Sim  
4             2025-04-29     NaN            Não  


In [67]:
limite_inativo = pd.Timestamp.today() - pd.Timedelta(days=9)

In [70]:
def classificar_status(row):
  if pd.isna(row['Mensagem Recebida']):
    return 'Não respondeu'
  elif 'agendar' in row['Mensagem Recebida'].lower():
    return 'Interessado'
  elif row['Data do Último Contato'] <limite_inativo:
    return 'Inativo'
  else:
    return 'Aguardando'

In [71]:
df['Status'] = df.apply(classificar_status, axis=1)

In [72]:
print(df[['Mensagem Recebida','Data do Último Contato', 'Status', 'Contato Ativo?']])

                Mensagem Recebida Data do Último Contato         Status  \
0                             NaN             2025-05-02  Não respondeu   
1                             NaN             2025-04-25  Não respondeu   
2       Não me respondeu ainda...             2025-04-24        Inativo   
3   Olá, qual o valor do serviço?             2025-05-07     Aguardando   
4                Tem vaga amanhã?             2025-04-29        Inativo   
..                            ...                    ...            ...   
95                            NaN             2025-05-07  Não respondeu   
96                 Quero agendar!             2025-04-24    Interessado   
97                 Quero agendar!             2025-04-25    Interessado   
98      Não me respondeu ainda...             2025-05-07     Aguardando   
99                 Quero agendar!             2025-05-01    Interessado   

   Contato Ativo?  
0             Não  
1             Não  
2             Não  
3             Sim  

In [74]:
from google.colab import files
df.to_csv('dfFormatado.csv', index=False)
files.download('dfFormatado.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
df.loc[df['Data do Último Contato'] < limite, ['Data do Último Contato','Mensagem Recebida', 'Status']]

,Data do Último Contato,Mensagem Recebida,Status
0,2025-05-02,NaN,Não respondeu
1,2025-04-25,NaN,Não respondeu
2,2025-04-24,Não me respondeu ainda...,Inativo
4,2025-04-29,Tem vaga amanhã?,Inativo
6,2025-04-27,"Olá, qual o valor do serviço?",Inativo
...,...,...,...
92,2025-04-27,NaN,Não respondeu
93,2025-05-04,Não me respondeu ainda...,Aguardando
96,2025-04-24,Quero agendar!,Interessado
97,2025-04-25,Quero agendar!,Interessado
